In [ ]:
import os
import qrcode
from flask import Flask, request, render_template, jsonify, send_file
from io import BytesIO
import traceback
import uuid

app = Flask(__name__)

# Initialize the in-memory database (empty dictionary for now)
customers = {}

# Ensure the 'qr_codes' directory exists
if not os.path.exists('qr_codes'):
    os.makedirs('qr_codes')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/register', methods=['POST'])
def register_customer():
    try:
        # Get form data
        name = request.form.get('name')
        email = request.form.get('email')
        phone = request.form.get('phone')

        if not name or not email or not phone:
            return jsonify({'error': 'All fields are required'}), 400

        # Generate a unique customer ID (UUID in this case)
        customer_id = str(uuid.uuid4())

        # Save the customer in the in-memory database
        customers[customer_id] = {'name': name, 'email': email, 'phone': phone}

        # Generate QR code for the customer ID
        qr = qrcode.make(customer_id)
        qr_file_path = f'qr_codes/{customer_id}.png'
        qr.save(qr_file_path)

        return render_template('registration_success.html', customer_id=customer_id, qr_code_url=f'/get_qr/{customer_id}')
    except Exception as e:
        traceback.print_exc()
        return jsonify({'error': 'An error occurred during registration'}), 500

@app.route('/get_qr/<customer_id>', methods=['GET'])
def get_qr_code(customer_id):
    try:
        if customer_id not in customers:
            return jsonify({'error': 'Customer not found'}), 404

        qr_file_path = f'qr_codes/{customer_id}.png'
        if not os.path.exists(qr_file_path):
            return jsonify({'error': 'QR code not found'}), 404

        return send_file(qr_file_path, mimetype='image/png')
    except Exception as e:
        traceback.print_exc()
        return jsonify({'error': 'An error occurred while fetching the QR code'}), 500

if __name__ == '__main__':
    app.run(debug=False)




 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1